In [1]:
import pandas as pd
import numpy as np
import psutil, os

In [2]:
p = psutil.Process(os.getpid())
p.nice(psutil.HIGH_PRIORITY_CLASS)

In [3]:
# Creamos diccionario de los equipos de la NFL
nfl_teams = pd.read_csv('Datos/nfl_teams.csv')
names = dict(zip(nfl_teams["team_name"], nfl_teams["team_id"]))

# Prueba
names["Jacksonville Jaguars"]

'JAX'

In [4]:
# Filtramos los datos por los años con mayor disponibilidad de los datos
nfl_scores = pd.read_csv('Datos/spreadspoke_scores.csv', encoding='utf-8')
nfl_scores = nfl_scores[nfl_scores["schedule_season"] >= 1999]
nfl_scores = nfl_scores.reset_index()
nfl_scores.head()

,index,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
0,7095,9/12/1999,1999,1,False,Atlanta Falcons,14,17,Minnesota Vikings,MIN,-4.0,49,Georgia Dome,False,72.0,0.0,NaN,indoor
1,7096,9/12/1999,1999,1,False,Chicago Bears,20,17,Kansas City Chiefs,KC,-3.0,38,Soldier Field,False,68.0,10.0,69.0,NaN
2,7097,9/12/1999,1999,1,False,Cleveland Browns,0,43,Pittsburgh Steelers,PIT,-6.0,37,FirstEnergy Stadium,False,70.0,10.0,54.0,NaN
3,7098,9/12/1999,1999,1,False,Green Bay Packers,28,24,Oakland Raiders,GB,-9.0,43,Lambeau Field,False,61.0,9.0,84.0,NaN
4,7099,9/12/1999,1999,1,False,Indianapolis Colts,31,14,Buffalo Bills,BUF,-3.0,45.5,RCA Dome,False,72.0,0.0,NaN,indoor


In [5]:
# Leemos los datos de las jugadas
plays_data = pd.read_csv("Datos/nflplays_1999_to_2022.csv", low_memory=False)
plays_data.head()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,out_of_bounds,home_opening_kickoff,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,xpass,pass_oe
0,35,1999_01_ARI_PHI,1999091200,PHI,ARI,REG,1,PHI,home,ARI,...,0,1,0.126818,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,60,1999_01_ARI_PHI,1999091200,PHI,ARI,REG,1,PHI,home,ARI,...,0,1,-0.561568,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,82,1999_01_ARI_PHI,1999091200,PHI,ARI,REG,1,PHI,home,ARI,...,0,1,-0.641717,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,103,1999_01_ARI_PHI,1999091200,PHI,ARI,REG,1,PHI,home,ARI,...,0,1,-0.723302,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,126,1999_01_ARI_PHI,1999091200,PHI,ARI,REG,1,PHI,home,ARI,...,0,1,0.212661,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
plays_data = plays_data[
    ["game_date", "season", "home_team", "touchdown", "posteam", "extra_point_result",
     "two_point_conv_result", "safety", "field_goal_result", "penalty_team", "penalty",
     "drive", "drive_first_downs", "drive_play_count", "drive_time_of_possession",
     "ydsnet", "qtr"]]
plays_data['game_date'] =  pd.to_datetime(plays_data['game_date'])

In [7]:
# Número de partidos a completar los datos
n_games = nfl_scores.shape[0]
print(n_games)

# Inicializamos arreglos para llenar datos
td_home = np.zeros(n_games)
td_away = np.zeros(n_games)
extra_home = np.zeros(n_games)
extra_away = np.zeros(n_games)
conv_home = np.zeros(n_games)
conv_away = np.zeros(n_games)
goal_home = np.zeros(n_games)
goal_away = np.zeros(n_games)
safety_home = np.zeros(n_games)
safety_away = np.zeros(n_games)
avgyd_home = np.zeros(n_games)
avgyd_away = np.zeros(n_games)
effdown_home = np.zeros(n_games)
effdown_away = np.zeros(n_games)
penalized_home = np.zeros(n_games)
penalized_away = np.zeros(n_games)
timepos_home = np.zeros(n_games)
timepos_away = np.zeros(n_games)
drives =  np.zeros(n_games)
overtime = np.zeros(n_games)

6421


In [8]:
def sum_minsec(times):
    total = 0
    for time in times:
        m, s = map(int, time.split(":"))
        total += 60*m + s
    return total

In [9]:
i = 0
for row in nfl_scores.to_numpy():
    team_home = names[row[5]]
    team_away = names[row[8]]
    plays = plays_data[(plays_data["season"] == row[2]) &
                       (plays_data["game_date"].dt.strftime('%#m/%#d/%Y') == row[1]) &
                       (plays_data["home_team"] == team_home)]

    # Touchdowns
    td_home[i] = len(plays[(plays["touchdown"] == 1) & (plays["posteam"] == team_home)])
    td_away[i] = len(plays[(plays["touchdown"] == 1) & (plays["posteam"] == team_away)])
    
    # Punto extra
    extra_home[i] = len(plays[(plays["extra_point_result"] == "good") & (plays["posteam"] == team_home)])
    extra_away[i] = len(plays[(plays["extra_point_result"] == "good") & (plays["posteam"] == team_away)])

    # Conversiones
    conv_home[i] = len(plays[(plays["two_point_conv_result"] == "success") & (plays["posteam"] == team_home)])
    conv_away[i] = len(plays[(plays["two_point_conv_result"] == "success") & (plays["posteam"] == team_away)])

    # Goles de campo
    goal_home[i] = len(plays[(plays["field_goal_result"] == "made") & (plays["posteam"] == team_home)])
    goal_away[i] = len(plays[(plays["field_goal_result"] == "made") & (plays["posteam"] == team_away)])
    
    # Safety
    safety_home[i] = len(plays[(plays["safety"] == 1) & (plays["posteam"] == team_away)])
    safety_away[i] = len(plays[(plays["safety"] == 1) & (plays["posteam"] == team_home)])
    
    # Penalizaciones
    penalized_home[i] = len(plays[(plays["penalty_team"] == team_home) & (plays["penalty"] == 1)])
    penalized_away[i] = len(plays[(plays["penalty_team"] == team_away) & (plays["penalty"] == 1)])

    # Tiempo Extra
    overtime[i] = int(plays["qtr"].max() == 5)

    # Número de series
    drives[i] = plays["drive"].max()

    last_rows = plays.drop_duplicates(
        subset = ["drive"])
    
    drives_home = last_rows[(last_rows["posteam"] == team_home) & (last_rows["drive_play_count"] > 0)]
    drives_away = last_rows[(last_rows["posteam"] == team_away) & (last_rows["drive_play_count"] > 0)]

    if len(drives_home) != 0 or len(drives_away) != 0:
        # Probabilidad de que se convierta en primero y 10
        effdown_home[i] = drives_home["drive_first_downs"].sum() / drives_home["drive_play_count"].sum()
        effdown_away[i] = drives_away["drive_first_downs"].sum() / drives_away["drive_play_count"].sum()

        # Porcentaje de posesión
        sec_home = sum_minsec(drives_home["drive_time_of_possession"].values)
        sec_away = sum_minsec(drives_away["drive_time_of_possession"].values)
        
        timepos_home[i] = sec_home/(sec_home + sec_away)
        timepos_away[i] = sec_away/(sec_home + sec_away)

        # Promedio de yardas avanzadas por jugada
        avgyd_home[i] = np.nanmean(drives_home["ydsnet"].values / drives_home["drive_play_count"].values)
        avgyd_away[i] = np.nanmean(drives_away["ydsnet"].values / drives_away["drive_play_count"].values)


    if i % 100 == 0:
        print(i, i/n_games, '%')
    i += 1

0 0.0 %


C:\Users\Iván\AppData\Local\Temp\ipykernel_15960\821122528.py:48: RuntimeWarning: invalid value encountered in double_scalars
  effdown_away[i] = drives_away["drive_first_downs"].sum() / drives_away["drive_play_count"].sum()
C:\Users\Iván\AppData\Local\Temp\ipykernel_15960\821122528.py:59: RuntimeWarning: Mean of empty slice
  avgyd_away[i] = np.nanmean(drives_away["ydsnet"].values / drives_away["drive_play_count"].values)


100 0.01557389814670612 %
200 0.03114779629341224 %
300 0.04672169444011836 %
400 0.06229559258682448 %
500 0.0778694907335306 %
600 0.09344338888023672 %
700 0.10901728702694284 %


KeyboardInterrupt: 

In [ ]:
columns = ["td_home","td_away","extra_home","extra_away","conv_home","conv_away",
           "goal_home","goal_away","safety_home","safety_away","avgyd_home",
           "avgyd_away","effdown_home","effdown_away","penalized_home","penalized_away",
           "timepos_home","timepos_away","drives","overtime"]
new_data = np.matrix([
    td_home, td_away, extra_home, extra_away, conv_home, conv_away, goal_home, 
    goal_away, safety_home, safety_away, avgyd_home, avgyd_away, effdown_home, 
    effdown_away, penalized_home, penalized_away, timepos_home, timepos_away, 
    drives , overtime 
]).T
nfl_scores_new = nfl_scores.values
new_data = np.hstack((nfl_scores_new, new_data))

new_data_nfl = pd.DataFrame(new_data, columns= (nfl_scores.columns.to_list()+columns))
new_data_nfl

,index,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,...,avgyd_home,avgyd_away,effdown_home,effdown_away,penalized_home,penalized_away,timepos_home,timepos_away,drives,overtime
0,13247,9/12/2022,2022,1,False,Seattle Seahawks,17,16,Denver Broncos,DEN,...,5.395833,8.414683,0.38,0.294118,5.0,12.0,0.439167,0.560833,16.0,0.0
1,13275,9/25/2022,2022,3,False,Seattle Seahawks,23,27,Atlanta Falcons,SEA,...,5.461611,6.029982,0.319444,0.428571,6.0,2.0,0.529444,0.470556,17.0,0.0
2,13324,10/16/2022,2022,6,False,Seattle Seahawks,19,9,Arizona Cardinals,ARI,...,2.424339,3.225049,0.271429,0.208333,6.0,6.0,0.487778,0.512222,23.0,0.0
3,13353,10/30/2022,2022,8,False,Seattle Seahawks,27,13,New York Giants,SEA,...,4.112821,2.212482,0.296875,0.212121,3.0,6.0,0.440556,0.559444,24.0,0.0
4,13408,11/27/2022,2022,12,False,Seattle Seahawks,34,40,Las Vegas Raiders,SEA,...,4.558639,NaN,0.338462,NaN,2.0,0.0,1.0,0.0,27.0,1.0
5,13437,12/11/2022,2022,14,False,Seattle Seahawks,24,30,Carolina Panthers,SEA,...,4.037421,3.812352,0.296296,0.32,3.0,6.0,0.345556,0.654444,21.0,0.0
6,13440,12/15/2022,2022,15,False,Seattle Seahawks,13,21,San Francisco 49ers,SF,...,2.397403,6.063657,0.253968,0.241935,8.0,7.0,0.449722,0.550278,23.0,0.0
7,13484,1/1/2023,2022,17,False,Seattle Seahawks,23,6,New York Jets,NYJ,...,5.038595,3.36063,0.266667,0.285714,5.0,2.0,0.560556,0.439444,23.0,0.0
8,13501,1/8/2023,2022,18,False,Seattle Seahawks,19,16,Los Angeles Rams,SEA,...,4.639044,NaN,0.293333,NaN,4.0,0.0,1.0,0.0,25.0,1.0


In [ ]:
new_data_nfl.to_csv("Datos/nfl_scores.csv")